In [3]:
import numpy as np
import cv2, os, pyautogui, time
from PIL import Image
import matplotlib.pyplot as plt
import warnings

from src.face_features import *
from src.tools import *

width, height= pyautogui.size()

In [2]:
cap = cv2.VideoCapture(0)
ret, frame = cap.read()
cv2.destroyAllWindows()
cap.release()

In [3]:
face_features = extract_image_features(frame)
# show_extraction_results(face_features)

In [2]:
from src.model import model

model.compile(optimizer='adam', loss="mse")
model.summary() 

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 64, 64, 3)    0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            (None, 64, 64, 3)    0                                            
__________________________________________________________________________________________________
input_3 (InputLayer)            (None, 64, 64, 3)    0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 54, 54, 32)   11648       input_1[0][0]                    
____________________________________________________________________________________________

In [4]:
X_left, X_right, X_face, X_mask, Y = [], [], [], [], []

In [ ]:
cap = cv2.VideoCapture(0)
# frame_width, frame_height = 320, 240
# cap.set(cv2.CAP_PROP_FRAME_WIDTH, frame_width)
# cap.set(cv2.CAP_PROP_FRAME_HEIGHT, frame_height)
cv2.namedWindow("window", cv2.WND_PROP_FULLSCREEN)
cv2.setWindowProperty("window",cv2.WND_PROP_FULLSCREEN,cv2.WINDOW_FULLSCREEN)
class GetOutOfLoop(Exception):
    pass


x, y = 0.5, 0.5
image = np.zeros((height, width, 3), np.uint8)
path = [[0,0], [1,0], [0,1], [1,1]]
for i in range(50):
    x_new, y_new = np.random.uniform(), np.random.uniform()
    while abs(x_new - path[-1][0]) + abs(y_new-path[-1][0]) < 1.0:
        x_new, y_new = np.random.uniform(), np.random.uniform()
    path.append([x_new, y_new])
np.random.shuffle(np.array(path))
    
try:
    for x_new, y_new in path:
        T = 2000
        n = 150
        for xc, yc in np.linspace([x, y], [x_new, y_new], n):
            image[:,:,:] = 215
            cv2.circle(image, (int(xc*width),int(yc*height)), radius=15, color=(35,35,35), thickness=-2)
            cv2.imshow("window", image)
            cv2.waitKey(T//n)
            ret, frame = cap.read()
            if ret:
                frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
                face_features = extract_image_features(frame)
                try:
                    face_features[1][0]; face_features[1][1]; face_features[0]; face_features[2]
                    X_left.append(cv2.resize(face_features[1][0], (img_size,img_size), interpolation = cv2.INTER_AREA)/255.0)
                    X_right.append(cv2.resize(face_features[1][1], (img_size,img_size), interpolation = cv2.INTER_AREA)/255.0)
                    X_face.append(cv2.resize(face_features[0], (img_size,img_size), interpolation = cv2.INTER_AREA)/255.0)
                    X_mask.append(face_features[2])
                    Y.append([xc,yc])
                except Exception as e:
                    print(e)
#                 img = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
#                 X_temp.append(np.expand_dims(img, -1))
                
#                 Y.append([xc,yc])
            if cv2.waitKey(1) & 0xFF == ord('q'):
                raise GetOutOfLoop
        x, y = x_new, y_new
except Exception as e:
    print(e)
finally:
    cv2.destroyAllWindows()
    cap.release()

In [271]:
from ipywidgets import IntProgress
from IPython.display import display

target_height, target_width = 32, 64
max_count = len(X_temp)

f = IntProgress(min=0, max=max_count) # instantiate the bar
display(f) # display the bar

X_left, X_right, X_face, X_mask, Y = [], [], [], [], []
for i, frame in enumerate(X_temp):
    face_features = extract_image_features(frame)
    try:
        face_features[1][0]; face_features[1][1]; face_features[0]; face_features[2]
        X_left.append(cv2.resize(face_features[1][0], (img_size,img_size), interpolation = cv2.INTER_AREA)/255.0)
        X_right.append(cv2.resize(face_features[1][1], (img_size,img_size), interpolation = cv2.INTER_AREA)/255.0)
        X_face.append(cv2.resize(face_features[0], (img_size,img_size), interpolation = cv2.INTER_AREA)/255.0)
        X_mask.append(face_features[2])
        Y.append(Y_temp[i])
    except:
        pass
    f.value += 1

IntProgress(value=0, max=2100)

In [31]:
X_left = np.expand_dims(np.array(X_left), -1)
X_right = np.expand_dims(np.array(X_right), -1)
X_face = np.expand_dims(np.array(X_face), -1)
X_mask = np.array(X_mask)
Y = np.array(Y)
print(X_left.shape, X_right.shape, X_face.shape, X_mask.shape, Y.shape5y6t)

In [277]:
X_left = np.concatenate([X_left, X_left_old])
X_right = np.concatenate([X_right, X_right_old])
X_face = np.concatenate([X_face, X_face_old])
X_mask = np.concatenate([X_mask, X_mask_old])
Y = np.concatenate([Y, Y_old])

In [257]:
X_left_old, X_right_old, X_face_old, X_mask_old, Y_old = X_left, X_right, X_face, X_mask, Y
X_left, X_right, X_face, X_mask, Y = [],[],[],[],[]

In [34]:
import pickle

pickle.dump([X_left, X_right, X_face, X_mask, Y], open("save2_c1.pkl", "wb"))

In [193]:
from sklearn.model_selection import train_test_split
train_set, test_set = train_test_split(list(range(len(Y))), train_size=0.9, shuffle=True)

In [133]:
X_train = np.hstack([np.array(X_left)[train_set], np.array(X_right)[train_set], np.array(X_face)[train_set], np.array(X_mask)[train_set]])
X_test = np.hstack([np.array(X_left)[test_set], np.array(X_right)[test_set], np.array(X_face)[test_set], np.array(X_mask)[test_set]])


In [202]:
X_train = [np.array(X_left)[train_set], np.array(X_right)[train_set], np.array(X_face)[train_set], np.array(X_mask)[train_set]]


In [ ]:
model.fit(X_train, np.array(Y)[train_set], batch_size=256, epochs=100)